# 101 LangGraph: Type Annotations

**Workshop**: LangGraph 101
**Duration**: ~5 minutes
**Difficulty**: Beginner

## Learning Objectives

By completing this notebook, you will:
- Understand Python dictionaries and their use as flexible data containers
- Learn TypedDict for defining structured dictionary schemas
- Master Union types for handling multiple possible types
- Use Optional for representing nullable values
- Understand Any for dynamic typing scenarios
- Apply Lambda functions for inline operations

## Prerequisites

- **Knowledge**: Basic Python syntax and data structures

## Table of Contents

1. [Introduction](#1-introduction)
2. [Python Dictionaries](#2-python-dictionaries)
3. [TypedDict for Structure](#3-typeddict-for-structure)
4. [Union and Optional Types](#4-union-and-optional-types)
5. [Any Type and Lambda Functions](#5-any-type-and-lambda-functions)
6. [Practical Examples](#6-practical-examples)
7. [Summary](#7-summary)

## 1. Introduction

Type annotations in Python provide a way to explicitly declare the types of variables, function parameters, and return values. While Python is dynamically typed, type hints help catch bugs early, improve code documentation, and enable better IDE support. In this notebook, we'll explore the fundamental type annotations you'll use when building LangGraph applications.

### Why Type Annotations Matter for Network Administrators

As a network administrator, you're familiar with structured configuration formats:
- **CLI configuration modes** require specific syntax and structure
- **YANG models** define strict schemas for network device configuration
- **ACLs** have precise syntax requirements (source, destination, action)

Type annotations in Python serve the same purpose - they define the "configuration schema" for your code, ensuring data has the correct structure before you use it.

Without proper type annotations:
- Code is harder to understand and maintain, especially in larger projects
- IDEs can't provide accurate autocomplete and error detection
- Bugs related to incorrect data types are caught at runtime instead of development time

With type annotations:
- Your code becomes self-documenting with clear contracts for what types are expected
- IDEs provide intelligent autocomplete and catch type errors before you run your code
- Team collaboration improves as everyone understands the expected data structures

### What We'll Build

In this notebook, we'll explore:
1. Python dictionaries and their role as flexible data containers
2. TypedDict for defining structured dictionary schemas
3. Union types for handling multiple possible types
4. Optional for nullable values
5. Any for dynamic typing when needed
6. Lambda functions for inline operations

Let's get started!

In [ ]:
import sys

# For type annotations, we only need the typing module (built-in)
# No external packages required!

print("✅ All required modules available")
print(f"   Python version: {sys.version.split()[0]}")
print("\nNote: The 'typing' module is built-in to Python 3.5+")

### 1.1 Import Dependencies

Let's import everything we'll need for this notebook.


In [ ]:
# Core imports for type annotations
from typing import TypedDict, Union, Optional, Any
from pprint import pprint

# Preview: SCM exception handling (you'll use this in later notebooks)
# These exceptions are raised by pan-scm-sdk when API operations fail
try:
    from scm.exceptions import (
        InvalidObjectError,      # Invalid configuration data
        NameNotUniqueError,      # Object name already exists
        ObjectNotPresentError,   # Object not found
        ReferenceNotZeroError,   # Object still in use (can't delete)
        MissingQueryParameterError  # Required parameter missing
    )
    print("✅ Imports successful (including SCM exceptions preview)")
    print("\n💡 SCM exceptions imported for preview - you'll use these")
    print("   extensively in later notebooks for proper error handling!")
except ImportError:
    print("✅ Imports successful")
    print("\n⚠️  Note: pan-scm-sdk not installed - that's OK for this foundations notebook!")
    print("   You'll install it before notebook 102 when you start building real workflows.")

---

## 2. Python Dictionaries

### What is a Dictionary?

A dictionary is Python's built-in key-value data structure that allows you to store and retrieve data using descriptive keys instead of numeric indices. Think of it like a real dictionary where you look up a word (key) to find its definition (value).

### Key Points

- **Flexible Structure**: Dictionaries can hold any type of data as values (strings, numbers, lists, even other dictionaries)
- **Dynamic**: You can add, modify, or remove keys at runtime without declaring a schema upfront
- **Unordered**: Dictionaries maintain insertion order (Python 3.7+), but are accessed by key, not position

### How It Works

Dictionaries use curly braces `{}` with key-value pairs separated by colons:

```python
# Creating a dictionary for network device configuration
device_config = {
    "hostname": "edge-router-01",
    "mgmt_ip": "192.168.1.1",
    "device_type": "router",
    "interfaces": 24,
    "uptime": 86400
}

# Accessing values
print(device_config["hostname"])  # Output: edge-router-01

# Adding new keys
device_config["location"] = "datacenter-a"

# Modifying values
device_config["uptime"] = 90000
```

### Strengths of Dictionaries

Dictionaries are awesome for many reasons:
- **Efficient data retrieval** based on unique keys
- **Flexible and easy to implement** for quick prototyping
- **Widely supported** across Python libraries and frameworks

### The Problem with Plain Dictionaries

However, there's a potential challenge: **it's difficult to ensure data has a particular structure**. This could be a huge problem in larger projects.

To put it simply: dictionaries don't check if the data is the correct data type or structure, and that could be the source of many logical errors in your project. In a large network automation workflow, this could be quite a headache to identify because it's such a small detail.

For example:
```python
# This is valid but probably wrong!
device_config = {
    "hostname": "edge-router-01",
    "mgmt_ip": 192168001001,  # Oops! Should be a string, not an integer
    "interfaces": "24"  # Oops! Should be an integer, not a string
}
# Python won't complain, but your code might break later
```

### Why This Matters in Network Automation

In network automation, a configuration error can have serious consequences:
- ❌ Wrong IP format → Device becomes unreachable
- ❌ Invalid zone name → Firewall rule doesn't apply
- ❌ Incorrect data type → Automation script fails mid-deployment

TypedDict acts as a **pre-deployment validation layer**, catching these errors during development instead of in production.

### The Solution: TypedDict

The solution to this problem is **TypedDict** (which we'll cover in the next section). TypedDict allows us to define a schema for our dictionaries, ensuring type safety while maintaining the flexibility of dictionaries.

### Why Dictionaries Matter for LangGraph

Dictionaries are the foundation for data structures in Python. While LangGraph uses TypedDict to define structured state schemas, TypedDict builds upon regular dictionaries. Understanding dictionaries is essential because at runtime, your typed state objects are actually dictionaries - TypedDict just adds type checking and structure on top.

### 2.1 Creating a Dictionary

In [ ]:
# Creating a dictionary for network device configuration
device_config = {
    "hostname": "edge-router-01",
    "mgmt_ip": "192.168.1.1",
    "device_type": "router",
    "interfaces": 24,
    "uptime": 86400
}

print("Device Configuration:")
pprint(device_config)

### 2.2 Accessing and Modifying Dictionary Values

In [ ]:
# Accessing values
print(f"Hostname: {device_config['hostname']}")
print(f"Management IP: {device_config['mgmt_ip']}")

# Adding new keys
device_config["location"] = "datacenter-a"
device_config["vendor"] = "Cisco"

# Modifying values
device_config["uptime"] = 90000

print("\nUpdated Device Configuration:")
pprint(device_config)

### 2.3 The Problem: No Type Checking

In [ ]:
# This is valid Python but has wrong data types!
problematic_config = {
    "hostname": "edge-router-01",
    "mgmt_ip": 192168001001,  # Oops! Should be a string, not an integer
    "device_type": "router",
    "interfaces": "24",  # Oops! Should be an integer, not a string
    "uptime": 86400
}

print("Problematic Configuration (no errors raised!):")
pprint(problematic_config)

# This will cause issues later when we try to use the data
print(f"\nTrying to use mgmt_ip as a string:")
try:
    print(f"IP address parts: {problematic_config['mgmt_ip'].split('.')}")
except AttributeError as e:
    print(f"❌ ERROR: {e}")
    print("\nThis is exactly the problem we're demonstrating!")
    print("The integer doesn't have a .split() method because it's not a string.")
    print("TypedDict would have caught this type mismatch during development!")
    print("Without type hints, Python happily accepts the wrong type and only fails at runtime.")

---

## 3. TypedDict for Structure

### What is a TypedDict?

TypedDict is a way to define a schema for dictionaries by explicitly declaring what keys should exist and what data type each value should be. It's implemented as a class, which makes it easy to reuse and provides excellent IDE support.

### Key Benefits

- **Type Safety**: Explicitly define what data types should be in your data structure, reducing runtime errors
- **Enhanced Readability**: Makes code self-documenting - anyone can see exactly what structure is expected
- **Better Debugging**: When something goes wrong, it's much easier to identify type mismatches
- **Critical for LangGraph**: TypedDict is used extensively in LangGraph to define state schemas

### How It Works

TypedDict is implemented as a class that inherits from `TypedDict`. Inside the class, you declare each key and its expected type using type annotations.

### Why TypedDict Matters for LangGraph

**This is extremely important**: TypedDict is used extensively in LangGraph to define the state that flows through your graph nodes. Every LangGraph application you build will use TypedDict to define what data your agent can access and modify. Don't worry - we'll cover states in detail in the next notebook, but understanding TypedDict now is essential.

### 3.1 Defining a TypedDict Schema

In [ ]:
# Define a TypedDict for network device configuration
class DeviceConfig(TypedDict):
    hostname: str
    mgmt_ip: str
    device_type: str
    interfaces: int
    uptime: int

print("✅ DeviceConfig schema defined")
print("\nSchema structure:")
for field_name, field_type in DeviceConfig.__annotations__.items():
    print(f"  - {field_name}: {field_type.__name__}")

### 3.1.1 Preview: Annotated Types with Reducers (Advanced Pattern)

**Don't worry about understanding this yet!** This is a preview of an advanced pattern you'll use in notebooks 108-111.

Modern LangGraph (2024-2025) uses `Annotated` types with reducers to enable automatic state management. Here's a quick preview:

In [ ]:
# Preview: Advanced pattern used in LangGraph agents (notebooks 108-111)
# You'll learn this in detail later - just getting familiar with the syntax now

from typing_extensions import Annotated
from langgraph.graph.message import add_messages

# This is how LangGraph agents manage conversation history automatically
class AgentState(TypedDict):
    """State schema for an AI agent that handles conversations"""
    messages: Annotated[list, add_messages]  # Automatically appends new messages!
    config_status: str  # Regular field, no reducer

print("✅ Annotated type preview loaded")
print("\n💡 What this does:")
print("   - Annotated[list, add_messages] means 'this is a list with special behavior'")
print("   - add_messages is a 'reducer' - it automatically appends new items to the list")
print("   - No manual list.append() needed - LangGraph handles it for you!")
print("\n📚 You'll use this pattern extensively in notebooks 108-111 for AI agents")
print("   For now, just know that TypedDict + Annotated = powerful state management!")

### 3.2 Creating an Instance of TypedDict

In [ ]:
# Creating an instance - looks just like a regular dictionary!
router: DeviceConfig = {
    "hostname": "core-router-01",
    "mgmt_ip": "10.0.0.1",
    "device_type": "router",
    "interfaces": 48,
    "uptime": 172800
}

print("Router Configuration:")
pprint(router)

# You can still access it like a regular dictionary
print(f"\nHostname: {router['hostname']}")
print(f"Interfaces: {router['interfaces']}")

### 3.3 Type Safety in Action

The beauty of TypedDict is that your IDE and type checkers (like mypy or pyright) can catch type errors before you even run the code. Let's see some examples:

In [ ]:
# This will show warnings in your IDE (though Python won't stop it at runtime)
# Your IDE would highlight these type mismatches!

switch: DeviceConfig = {
    "hostname": "access-switch-01",
    "mgmt_ip": "10.0.1.10",
    "device_type": "switch",
    "interfaces": 24,
    "uptime": 259200
}

print("✅ Correct types - IDE is happy!")
pprint(switch)

# Example of what would trigger IDE warnings (uncomment to see):
# wrong_device: DeviceConfig = {
#     "hostname": "firewall-01",
#     "mgmt_ip": 10001010,  # IDE Warning: Expected str, got int
#     "device_type": "firewall",
#     "interfaces": "8",  # IDE Warning: Expected int, got str
#     "uptime": 86400
# }

print("\n💡 Tip: TypedDict helps catch errors during development, not just at runtime!")

### 3.4 Using TypedDict in Functions

TypedDict really shines when used in function signatures, making it crystal clear what structure your functions expect:

In [ ]:
def configure_device(config: DeviceConfig) -> str:
    """
    Configure a network device with the provided configuration.
    
    Args:
        config: DeviceConfig dictionary containing device settings
        
    Returns:
        Configuration status message
    """
    return f"Configuring {config['hostname']} ({config['device_type']}) at {config['mgmt_ip']}"

def get_device_uptime_hours(config: DeviceConfig) -> float:
    """
    Calculate device uptime in hours.
    
    Args:
        config: DeviceConfig dictionary containing device settings
        
    Returns:
        Uptime in hours
    """
    return config['uptime'] / 3600

# Use the functions
result = configure_device(router)
print(result)

uptime_hours = get_device_uptime_hours(router)
print(f"Uptime: {uptime_hours:.2f} hours")

# Your IDE knows exactly what keys are available and their types!
# This provides excellent autocomplete support

### 3.5 Nested TypedDict for Complex Configurations

Real-world network configurations often have nested structures. TypedDict handles this beautifully:

In [ ]:
# Define nested TypedDicts for a firewall policy
class FirewallRule(TypedDict):
    name: str
    source: str
    destination: str
    port: int
    action: str

class FirewallPolicy(TypedDict):
    policy_name: str
    device: str
    rules: list[FirewallRule]
    enabled: bool

# Create a firewall policy with nested rules
policy: FirewallPolicy = {
    "policy_name": "datacenter-ingress",
    "device": "firewall-01",
    "enabled": True,
    "rules": [
        {
            "name": "allow-https",
            "source": "0.0.0.0/0",
            "destination": "10.0.0.0/24",
            "port": 443,
            "action": "allow"
        },
        {
            "name": "allow-ssh",
            "source": "192.168.1.0/24",
            "destination": "10.0.0.0/24",
            "port": 22,
            "action": "allow"
        }
    ]
}

print("Firewall Policy Configuration:")
pprint(policy)

print(f"\nPolicy '{policy['policy_name']}' has {len(policy['rules'])} rules")

print("\n" + "="*60)
print("Real-World SCM Integration Pattern")
print("="*60)

print("""
These TypedDict patterns mirror the structure you'll use with the pan-scm-sdk:

Example from docs/examples/address_objects.py:
    address_config = {
        "name": "web-server-01",
        "ip_netmask": "192.168.1.100/32",
        "folder": "Texas",
        "tag": ["Automation"]
    }

In upcoming notebooks, you'll use TypedDict to define state schemas that 
interact with the Strata Cloud Manager API using these exact patterns.

💡 The nested TypedDict structure you just learned is the foundation for
   building complex SCM configurations in LangGraph workflows!
""")

### 3.6 Real-World SCM Address Object Structures

Now let's apply what we've learned to actual Palo Alto Networks Strata Cloud Manager configurations. These TypedDict schemas mirror the exact structure you'll use with the pan-scm-sdk.

In [ ]:
# Define TypedDict schemas based on SCM address object API
# Reference: docs/examples/address_objects.py
# - Lines 9-15: IP/Netmask address configuration
# - Lines 21-26: FQDN address configuration  
# - Lines 32-37: IP Range address configuration
class AddressObjectNetmask(TypedDict):
    """IP/Netmask address object - matches SCM API structure"""
    name: str
    ip_netmask: str
    folder: str
    description: Optional[str]
    tag: Optional[list[str]]

class AddressObjectFQDN(TypedDict):
    """FQDN address object - matches SCM API structure"""
    name: str
    fqdn: str
    folder: str
    description: Optional[str]
    tag: Optional[list[str]]

class AddressObjectRange(TypedDict):
    """IP Range address object - matches SCM API structure"""
    name: str
    ip_range: str
    folder: str
    description: Optional[str]
    tag: Optional[list[str]]

# Create real SCM address configurations (matching docs/examples/address_objects.py)
netmask_address: AddressObjectNetmask = {
    "name": "internal_network",
    "ip_netmask": "192.168.1.0/24",
    "folder": "Texas",
    "description": "Internal network segment",
    "tag": ["Python", "Automation"]
}

fqdn_address: AddressObjectFQDN = {
    "name": "example_site",
    "fqdn": "example.com",
    "folder": "Texas",
    "description": "Example website",
    "tag": None  # No tags for this address
}

range_address: AddressObjectRange = {
    "name": "dhcp_pool",
    "ip_range": "192.168.1.100-192.168.1.200",
    "folder": "Texas",
    "description": "DHCP address pool",
    "tag": ["DHCP", "Infrastructure"]
}

print("SCM Address Object Configurations:")
print("\n1. IP/Netmask Address (from address_objects.py lines 9-15):")
pprint(netmask_address)

print("\n2. FQDN Address (from address_objects.py lines 21-26):")
pprint(fqdn_address)

print("\n3. IP Range Address (from address_objects.py lines 32-37):")
pprint(range_address)

print("\n" + "="*60)
print("💡 Key Insight: TypedDict Prevents Configuration Errors")
print("="*60)
print("""
These TypedDict definitions ensure:
✅ Required fields are never missing (name, ip_netmask/fqdn/ip_range, folder)
✅ Field types are correct (strings for names, lists for tags)
✅ IDE autocomplete shows available fields
✅ Type checkers catch errors BEFORE deployment

Without TypedDict, you might accidentally create:
❌ Missing 'folder' field → API rejects the request
❌ 'tag' as string instead of list → API error
❌ 'ip_netmask' with invalid format → Configuration fails

💡 See docs/examples/address_objects.py for real pan-scm-sdk usage patterns!
📚 You can use these exact structures with:
   client.address.create(netmask_address)
   client.address.create(fqdn_address)
   client.address.create(range_address)
""")

### 3.7 Demonstrating Configuration Error Prevention

Let's see how TypedDict helps catch common configuration mistakes that would otherwise cause runtime failures.

In [ ]:
def validate_address_config(config: dict) -> tuple[bool, str]:
    """
    Validate an address configuration dictionary.
    Returns (is_valid, error_message)
    """
    # Check required fields
    required_fields = ['name', 'folder']
    for field in required_fields:
        if field not in config:
            return False, f"Missing required field: '{field}'"
    
    # Check that exactly one address type is specified
    address_types = ['ip_netmask', 'fqdn', 'ip_range']
    present_types = [t for t in address_types if t in config]
    
    if len(present_types) == 0:
        return False, "Must specify one address type: ip_netmask, fqdn, or ip_range"
    if len(present_types) > 1:
        return False, f"Multiple address types specified: {present_types}"
    
    # Check tag type if present
    if 'tag' in config and config['tag'] is not None:
        if not isinstance(config['tag'], list):
            return False, f"Field 'tag' must be a list, got {type(config['tag']).__name__}"
    
    return True, "Configuration valid"

print("Testing Configuration Validation:\n")

# ❌ Example 1: Missing folder field
bad_config_1 = {
    "name": "web_server",
    "ip_netmask": "10.0.1.100/32"
}
is_valid, error = validate_address_config(bad_config_1)
print(f"Config 1: {bad_config_1}")
print(f"Result: {'✅ Valid' if is_valid else f'❌ Invalid - {error}'}\n")

# ❌ Example 2: Wrong type for tags (string instead of list)
bad_config_2 = {
    "name": "db_server",
    "ip_netmask": "10.0.2.100/32",
    "folder": "Texas",
    "tag": "Database"  # Should be a list!
}
is_valid, error = validate_address_config(bad_config_2)
print(f"Config 2: {bad_config_2}")
print(f"Result: {'✅ Valid' if is_valid else f'❌ Invalid - {error}'}\n")

# ❌ Example 3: Multiple address types specified
bad_config_3 = {
    "name": "confused_address",
    "ip_netmask": "10.0.3.0/24",
    "fqdn": "example.com",  # Can't have both!
    "folder": "Texas"
}
is_valid, error = validate_address_config(bad_config_3)
print(f"Config 3: {bad_config_3}")
print(f"Result: {'✅ Valid' if is_valid else f'❌ Invalid - {error}'}\n")

# ✅ Example 4: Correct configuration
good_config: AddressObjectNetmask = {
    "name": "app_server",
    "ip_netmask": "10.0.4.100/32",
    "folder": "Texas",
    "description": "Application server",
    "tag": ["Production", "WebApp"]
}
is_valid, error = validate_address_config(good_config)
print(f"Config 4: {good_config}")
print(f"Result: {'✅ Valid' if is_valid else f'❌ Invalid - {error}'}\n")

print("="*60)
print("💡 TypedDict + IDE = Errors Caught During Development")
print("="*60)
print("""
With TypedDict and a good IDE (VS Code, PyCharm), you would see:
- Red squiggly lines under incorrect field types
- Autocomplete suggesting valid fields
- Warnings about missing required fields
- Type mismatches highlighted BEFORE running code

This is the power of type annotations in production automation!
""")

---

## 4. Union and Optional Types

### What is Union?

Union is a type annotation that indicates a value can be one of several different types. It provides flexibility while still maintaining type safety by explicitly declaring which types are acceptable.

### Key Points About Union

- **Multiple Valid Types**: Allows a parameter or variable to accept multiple specific types
- **Type Safety**: Provides hints to catch incorrect usage - only the declared types are valid
- **Extensively Used**: The makers of LangChain and LangGraph use Union throughout their libraries
- **Flexible Yet Controlled**: Gives you flexibility while preventing completely wrong types

### What is Optional?

Optional is similar to Union but specifically for values that can be either a specific type or `None`. It's syntactic sugar for `Union[Type, None]`.

### When to Use Each

- **Union**: When a value can be one of several different types (e.g., `Union[int, float]`)
- **Optional**: When a value might be absent/None (e.g., `Optional[str]` = `Union[str, None]`)

### 4.1 Union - Accepting Multiple Types

In [ ]:
def calculate_bandwidth(value: Union[int, float]) -> float:
    """
    Calculate bandwidth in Gbps.
    
    The value can be either an integer or float representing Mbps,
    and this function converts it to Gbps.
    
    Args:
        value: Bandwidth in Mbps (can be int or float)
        
    Returns:
        Bandwidth in Gbps
    """
    return value / 1000

# These all work fine - both int and float are accepted
bandwidth1 = calculate_bandwidth(1000)
print(f"1000 Mbps = {bandwidth1} Gbps")

bandwidth2 = calculate_bandwidth(1234.5)
print(f"1234.5 Mbps = {bandwidth2} Gbps")

bandwidth3 = calculate_bandwidth(500)
print(f"500 Mbps = {bandwidth3} Gbps")

# This would fail type checking (uncomment to see):
# bandwidth4 = calculate_bandwidth("1000")  # IDE Warning: Expected int | float, got str
# This would cause an error at runtime!

### 4.2 Union with More Complex Types

In [ ]:
def process_device_identifier(device_id: Union[str, int]) -> str:
    """
    Process a device identifier which could be a hostname or device ID number.
    
    Args:
        device_id: Either a hostname (str) or device ID number (int)
        
    Returns:
        Formatted device identifier string
    """
    if isinstance(device_id, str):
        return f"Device hostname: {device_id}"
    else:
        return f"Device ID: {device_id:05d}"

# Works with string hostname
result1 = process_device_identifier("edge-router-01")
print(result1)

# Works with integer device ID
result2 = process_device_identifier(42)
print(result2)

# Works with another string
result3 = process_device_identifier("core-switch-01")
print(result3)

print("\n💡 Union allows flexibility while maintaining type safety!")

### 4.3 Optional - Handling Nullable Values

In [ ]:
def create_welcome_message(username: Optional[str] = None) -> str:
    """
    Create a welcome message for a network admin.
    
    Args:
        username: Optional username. If None, uses a generic greeting.
        
    Returns:
        Welcome message string
    """
    if username is not None:
        return f"Hi there, {username}! Welcome to the network management system."
    else:
        return "Hi there, random person! Welcome to the network management system."

# With a username
message1 = create_welcome_message("Alice")
print(message1)

# Without a username (None) - goes to the else statement
message2 = create_welcome_message()
print(message2)

# Explicitly passing None
message3 = create_welcome_message(None)
print(message3)

print("\n⚠️  Important: Optional[str] means ONLY str or None!")
print("It CANNOT be an integer, boolean, float, or any other type.")
print("This would fail type checking:")
print("# create_welcome_message(123)        # IDE Warning: Expected str | None, got int")
print("# create_welcome_message(True)       # IDE Warning: Expected str | None, got bool")
print("# create_welcome_message(3.14)       # IDE Warning: Expected str | None, got float")

### 4.4 Optional in Network Configurations

In [ ]:
class InterfaceConfig(TypedDict):
    name: str
    ip_address: str
    subnet_mask: str
    description: Optional[str]  # Description is optional
    vlan_id: Optional[int]      # VLAN ID is optional

# Interface with all fields
interface1: InterfaceConfig = {
    "name": "GigabitEthernet0/1",
    "ip_address": "192.168.1.1",
    "subnet_mask": "255.255.255.0",
    "description": "Uplink to core router",
    "vlan_id": 100
}

# Interface with optional fields missing (set to None)
interface2: InterfaceConfig = {
    "name": "GigabitEthernet0/2",
    "ip_address": "192.168.2.1",
    "subnet_mask": "255.255.255.0",
    "description": None,
    "vlan_id": None
}

print("Interface 1 (all fields):")
pprint(interface1)

print("\nInterface 2 (optional fields are None):")
pprint(interface2)

print("\n💡 Optional is perfect for configurations where some fields might not always be present!")

### 4.5 Combining Union and Optional

In [ ]:
def format_port_number(port: Union[int, str, None]) -> str:
    """
    Format a port number for display.
    Can accept int, str, or None.
    
    Args:
        port: Port number as int, str, or None
        
    Returns:
        Formatted port string
    """
    if port is None:
        return "Port: Not specified"
    elif isinstance(port, int):
        return f"Port: {port}"
    else:
        return f"Port: {port} (from string)"

# All of these work!
print(format_port_number(443))
print(format_port_number("8080"))
print(format_port_number(None))

print("\n💡 You can combine Union and Optional for maximum flexibility!")
print("Note: Union[int, str, None] is the same as Optional[Union[int, str]]")

---

## 5. Any Type and Lambda Functions

### What is Any?

The `Any` type is the easiest type annotation to understand: it literally means the value could be **anything**. It could be any data type or structure - no restrictions whatsoever.

### Key Points About Any

- **No Type Restrictions**: Accepts absolutely any data type
- **Maximum Flexibility**: Useful when you truly don't know or care about the type
- **Use Sparingly**: Overusing `Any` defeats the purpose of type annotations
- **Escape Hatch**: Good for working with dynamic data or third-party code without types

### When to Use Any

- Working with truly dynamic data from external sources
- Migrating legacy code to use type hints gradually
- Interfacing with libraries that don't have type annotations
- When the type is genuinely unknowable at development time

### When NOT to Use Any

- When you know the possible types (use Union instead)
- When you want type safety (use specific types or TypedDict)
- As a shortcut to avoid thinking about proper types

### 5.1 Basic Any Example

In [ ]:
def print_value(value: Any) -> None:
    """
    Print any value - literally anything is allowed!
    
    Args:
        value: Can be any type whatsoever
    """
    print(f"Value: {value}")
    print(f"Type: {type(value).__name__}")

# Everything and anything is allowed!
print_value("Device configuration updated")
print()
print_value(42)
print()
print_value(3.14159)
print()
print_value(True)
print()
print_value(["router", "switch", "firewall"])
print()
print_value({"hostname": "router-01", "ip": "10.0.0.1"})

print("\n✅ All types accepted - no warnings or errors!")

### 5.2 Practical Use Case: Logging Network Events

In [ ]:
def log_network_event(event_type: str, event_data: Any) -> None:
    """
    Log a network event with arbitrary data.
    
    Event data could be anything - a string, number, dict, list, etc.
    
    Args:
        event_type: Type of event (e.g., "config_change", "interface_down")
        event_data: Any data associated with the event
    """
    print(f"[{event_type.upper()}] Event Data:")
    pprint(event_data)
    print()

# Different types of event data
log_network_event("interface_down", "GigabitEthernet0/1")

log_network_event("config_change", {
    "device": "router-01",
    "changes": ["hostname", "interface"],
    "timestamp": 1234567890
})

log_network_event("bandwidth_alert", 95.5)

log_network_event("devices_offline", ["switch-01", "switch-02", "router-03"])

print("💡 Any is useful when the data structure is truly unpredictable!")

### 5.3 When to Prefer Specific Types Over Any

In [ ]:
# ❌ BAD: Using Any when you know the possible types
def calculate_metric_bad(value: Any) -> float:
    return value * 100

# ✅ GOOD: Using Union when you know the possible types
def calculate_metric_good(value: Union[int, float]) -> float:
    return value * 100

# ❌ BAD: Using Any when you know the structure
def process_device_bad(device: Any) -> str:
    return f"Processing {device['hostname']}"

# ✅ GOOD: Using TypedDict when you know the structure
def process_device_good(device: DeviceConfig) -> str:
    return f"Processing {device['hostname']}"

print("✅ Use specific types when possible:")
print("   - Union when you know the possible types")
print("   - TypedDict when you know the structure")
print("   - Optional when a value might be None")
print()
print("⚠️  Reserve Any for truly dynamic/unknown data:")
print("   - External API responses without schemas")
print("   - User input that could be anything")
print("   - Legacy code you're gradually typing")

---

## 6. Lambda Functions

### What is a Lambda Function?

Lambda functions are anonymous, inline functions that provide a shortcut for writing small, single-expression functions. They make code more efficient and concise, especially when used with functions like `map()`, `filter()`, and `sorted()`.

### Key Points About Lambda

- **Inline Definition**: Define a function in one line without using `def`
- **Anonymous**: No need to give the function a name
- **Single Expression**: Limited to one expression (no statements or multiple lines)
- **Efficient**: Perfect for simple operations that don't need a full function definition

### Syntax

```python
lambda arguments: expression
```

### When to Use Lambda

- Simple transformations in `map()`, `filter()`, `sorted()`
- One-time use functions that don't need a name
- Callbacks or key functions
- Making code more concise and readable

### When NOT to Use Lambda

- Complex logic requiring multiple lines
- Functions you'll reuse in multiple places (use `def` instead)
- When it reduces readability

### 6.1 Lambda vs Regular Function

In [ ]:
# Regular function approach
def calculate_vlan_priority(vlan_id: int) -> int:
    """Calculate priority for a VLAN ID"""
    return vlan_id * 10

priority1 = calculate_vlan_priority(100)
print(f"Regular function - VLAN 100 priority: {priority1}")

# Lambda function approach - same logic, one line!
calculate_vlan_priority_lambda = lambda vlan_id: vlan_id * 10

priority2 = calculate_vlan_priority_lambda(100)
print(f"Lambda function - VLAN 100 priority: {priority2}")

print("\n✅ Both produce the same result!")
print("Lambda is just a shortcut for simple, one-line functions.")

### 6.2 Lambda with map() - Transforming Lists

In [ ]:
# Port numbers that need to be converted to service names
ports = [22, 80, 443, 8080]

# Beginner approach - using a for loop
ports_formatted_beginner = []
for port in ports:
    ports_formatted_beginner.append(f"port-{port}")

print("Beginner approach (for loop):")
print(ports_formatted_beginner)

# Advanced approach - using map() with lambda
ports_formatted_advanced = list(map(lambda port: f"port-{port}", ports))

print("\nAdvanced approach (map + lambda):")
print(ports_formatted_advanced)

print("\n✅ Same result, but lambda + map is more concise and efficient!")
print("map() applies the lambda function to each element in the list.")

### 6.3 Lambda with map() - Calculating Interface Bandwidth

In [ ]:
# Interface throughput in bytes per second
interface_throughput_bytes = [125000, 62500, 31250, 15625]

# More practical example: Convert bytes to bits for bandwidth calculation
# 1 byte = 8 bits
bandwidth_bits = list(map(lambda bytes_per_sec: bytes_per_sec * 8, interface_throughput_bytes))

print("\nBandwidth in bits/sec:", bandwidth_bits)
print("Bandwidth in Kbps:", list(map(lambda bits: bits / 1000, bandwidth_bits)))

print("\n💡 Lambda with map() is much more efficient than writing a for loop!")
print("The map function applies the lambda to each value in the list.")

### 6.4 Lambda with filter() - Filtering Network Data

In [ ]:
# List of interface utilization percentages
interface_utilization = [45, 78, 92, 34, 88, 95, 23, 67]

# Filter to find only high utilization interfaces (>= 80%)
high_utilization = list(filter(lambda util: util >= 80, interface_utilization))

print("All interface utilization:", interface_utilization)
print("High utilization (>= 80%):", high_utilization)

# Filter VLAN IDs to find only production VLANs (100-199)
vlan_ids = [10, 50, 100, 150, 200, 250, 175, 999]
production_vlans = list(filter(lambda vlan: 100 <= vlan < 200, vlan_ids))

print("\nAll VLAN IDs:", vlan_ids)
print("Production VLANs (100-199):", production_vlans)

print("\n💡 Lambda with filter() keeps only elements that match your condition!")

### 6.5 Lambda with sorted() - Sorting Network Devices

In [ ]:
# List of devices with their priority levels
devices = [
    {"hostname": "router-01", "priority": 10},
    {"hostname": "switch-05", "priority": 3},
    {"hostname": "firewall-01", "priority": 8},
    {"hostname": "router-02", "priority": 10},
    {"hostname": "switch-01", "priority": 5}
]

# Sort by priority (highest first)
sorted_by_priority = sorted(devices, key=lambda device: device["priority"], reverse=True)

print("Devices sorted by priority (highest first):")
for device in sorted_by_priority:
    print(f"  {device['hostname']}: priority {device['priority']}")

# Sort by hostname alphabetically
sorted_by_name = sorted(devices, key=lambda device: device["hostname"])

print("\nDevices sorted by hostname:")
for device in sorted_by_name:
    print(f"  {device['hostname']}: priority {device['priority']}")

print("\n💡 Lambda with sorted() is perfect for custom sorting logic!")

### 6.6 Lambda Functions - Key Takeaways

Lambda functions are powerful shortcuts that make your code more concise and efficient. They're especially useful when combined with `map()`, `filter()`, and `sorted()`.

**Remember:**
- No need to memorize the syntax - just understand the concept
- Use lambda for simple, one-line operations
- Lambda makes code more efficient than writing for loops
- You'll see lambda functions throughout Python and LangGraph code

**Common patterns:**
- `map(lambda x: transform(x), list)` - Transform each element
- `filter(lambda x: condition(x), list)` - Keep elements matching a condition  
- `sorted(list, key=lambda x: x['field'])` - Custom sorting logic

### 6.7 Advanced Exercise: Security Rule Structures

Let's apply everything we've learned to create TypedDict schemas for Palo Alto Networks security rules - a more complex, real-world scenario.

In [ ]:
# Define comprehensive TypedDict schemas for security rules
# Reference: docs/examples/security_policy.py (lines 9-20, 26-39)
class SecurityRuleBase(TypedDict):
    """Base security rule structure for SCM - matches pan-scm-sdk API"""
    name: str
    folder: str
    from_: list[str]  # SCM API uses 'from_' not 'from_zones'
    to_: list[str]  # SCM API uses 'to_' not 'to_zones'
    source: list[str]  # SCM API uses 'source' not 'source_addresses'
    destination: list[str]  # SCM API uses 'destination' not 'destination_addresses'
    application: list[str]  # SCM API uses 'application' not 'applications'
    service: list[str]  # SCM API uses 'service' not 'services'
    action: str  # "allow", "deny", "drop", "reset-client", "reset-server", "reset-both"

class SecurityRuleFull(TypedDict):
    """Complete security rule with all options - matches pan-scm-sdk API

    Reference: docs/examples/security_policy.py
    - Lines 9-20: Basic allow rule with log_end
    - Lines 26-39: Secure rule with profile_setting
    """
    name: str
    folder: str
    from_: list[str]
    to_: list[str]
    source: list[str]
    destination: list[str]
    application: list[str]
    service: list[str]
    action: str
    description: Optional[str]
    tag: Optional[list[str]]
    disabled: Optional[bool]
    log_start: Optional[bool]
    log_end: Optional[bool]
    profile_setting: Optional[dict]

# Example 1: Basic allow rule for web traffic (matches security_policy.py lines 9-20)
web_allow_rule: SecurityRuleFull = {
    "name": "Allow-Web-Traffic",
    "folder": "Texas",
    "from_": ["trust"],
    "to_": ["untrust"],
    "source": ["internal-net"],
    "destination": ["any"],
    "application": ["web-browsing", "ssl"],
    "service": ["application-default"],
    "action": "allow",
    "description": "Allow internal users to browse the web",
    "tag": ["Production", "Web"],
    "disabled": False,
    "log_start": False,
    "log_end": True,
    "profile_setting": None
}

# Example 2: Secure rule with security profiles (matches security_policy.py lines 26-39)
secure_web_rule: SecurityRuleFull = {
    "name": "Secure-Web-Access",
    "folder": "Texas",
    "from_": ["trust"],
    "to_": ["untrust"],
    "source": ["internal-net"],
    "destination": ["any"],
    "application": ["web-browsing", "ssl"],
    "service": ["application-default"],
    "action": "allow",
    "description": "Web access with security profiles applied",
    "tag": ["Production", "Security"],
    "disabled": False,
    "log_start": False,
    "log_end": True,
    "profile_setting": {"group": ["best-practice"]}
}

# Example 3: Deny rule for blocking specific applications
app_deny_rule: SecurityRuleBase = {
    "name": "Block-P2P-Apps",
    "folder": "Texas",
    "from_": ["trust"],
    "to_": ["untrust"],
    "source": ["any"],
    "destination": ["any"],
    "application": ["bittorrent", "skype"],
    "service": ["application-default"],
    "action": "deny"
}

print("Security Rule Examples:\n")
print("1. Web Allow Rule (Basic - from security_policy.py lines 9-20):")
pprint(web_allow_rule)

print("\n2. Secure Web Rule (With Profiles - from security_policy.py lines 26-39):")
pprint(secure_web_rule)

print("\n3. Application Deny Rule (Minimal):")
pprint(app_deny_rule)

print("\n" + "="*60)
print("Using Lambda to Filter and Process Security Rules")
print("="*60)

# Create a collection of security rules
all_rules = [web_allow_rule, secure_web_rule, app_deny_rule]

# Filter only "allow" rules using lambda
allow_rules = list(filter(lambda rule: rule["action"] == "allow", all_rules))
print(f"\nAllow rules: {len(allow_rules)}")
for rule in allow_rules:
    print(f"  - {rule['name']}: {rule['from_']} → {rule['to_']}")

# Filter rules with tags containing "Production"
production_rules = list(filter(
    lambda rule: rule.get("tag") and "Production" in rule["tag"],
    all_rules
))
print(f"\nProduction rules: {len(production_rules)}")
for rule in production_rules:
    print(f"  - {rule['name']}: Tagged with {rule['tag']}")

# Filter rules with security profiles applied
profile_rules = list(filter(
    lambda rule: rule.get("profile_setting") is not None,
    all_rules
))
print(f"\nRules with security profiles: {len(profile_rules)}")
for rule in profile_rules:
    print(f"  - {rule['name']}: Profile = {rule.get('profile_setting')}")

# Get all unique zones using lambda and map
all_zones = set()
for rule in all_rules:
    all_zones.update(rule["from_"])
    all_zones.update(rule["to_"])

print(f"\nAll zones referenced in rules: {sorted(all_zones)}")

# Count rules by action type
from collections import Counter
action_counts = Counter(map(lambda rule: rule["action"], all_rules))
print(f"\nRules by action:")
for action, count in action_counts.items():
    print(f"  {action}: {count}")

# Use lambda with sorted() to order rules by name
sorted_rules = sorted(all_rules, key=lambda rule: rule["name"])
print(f"\nRules sorted by name:")
for rule in sorted_rules:
    print(f"  - {rule['name']}")

print("\n💡 This demonstrates how TypedDict + Lambda creates powerful,")
print("   type-safe automation for managing security configurations!")
print("\n📚 Reference: These structures match docs/examples/security_policy.py")
print("   exactly - use them in your LangGraph workflows!")

### 6.8 Hands-On Exercise: Build Your Own Configuration

Now it's your turn! Try creating your own security rule configuration using the TypedDict schemas we've defined.

In [ ]:
print("Exercise: Create Your Own Security Rule")
print("="*60)
print("""
Task: Create a security rule with the following requirements:
1. Name: "Allow-SSH-Admin"
2. Allow SSH access from the 'management' zone to 'datacenter' zone
3. Source: Management subnet (192.168.100.0/24)
4. Destination: Server subnet (10.0.10.0/24)
5. Application: ssh
6. Service: application-default
7. Action: allow
8. Add tags: ["Management", "SSH", "Admin"]
9. Enable logging at session end
10. Add description explaining the rule purpose

Try creating this configuration below:
""")

# YOUR SOLUTION HERE - Uncomment and complete:
# my_security_rule: SecurityRuleFull = {
#     "name": "...",
#     "folder": "...",
#     # ... add remaining fields
# }

# Here's a working solution (revealed):
admin_ssh_rule: SecurityRuleFull = {
    "name": "Allow-SSH-Admin",
    "folder": "Texas",
    "from_": ["management"],
    "to_": ["datacenter"],
    "source": ["192.168.100.0/24"],
    "destination": ["10.0.10.0/24"],
    "application": ["ssh"],
    "service": ["application-default"],
    "action": "allow",
    "description": "Allow SSH access from management subnet to datacenter servers for administrative access",
    "tag": ["Management", "SSH", "Admin"],
    "disabled": False,
    "log_start": False,
    "log_end": True,
    "profile_setting": None
}

print("\n✅ Solution:")
pprint(admin_ssh_rule)

print("\n" + "="*60)
print("Validation Check")
print("="*60)

# Validate the configuration
def validate_security_rule(rule: dict) -> None:
    """Validate a security rule configuration"""
    checks = []
    
    # Check required fields (using correct SCM API field names)
    required = ["name", "folder", "from_", "to_", "source", 
                "destination", "application", "service", "action"]
    for field in required:
        if field in rule:
            checks.append(f"✅ Has required field: {field}")
        else:
            checks.append(f"❌ Missing required field: {field}")
    
    # Check list fields are actually lists
    list_fields = ["from_", "to_", "source", 
                   "destination", "application", "service"]
    for field in list_fields:
        if field in rule:
            if isinstance(rule[field], list):
                checks.append(f"✅ {field} is a list with {len(rule[field])} item(s)")
            else:
                checks.append(f"❌ {field} should be a list, got {type(rule[field]).__name__}")
    
    # Check action is valid
    valid_actions = ["allow", "deny", "drop", "reset-client", "reset-server", "reset-both"]
    if rule.get("action") in valid_actions:
        checks.append(f"✅ Valid action: {rule['action']}")
    else:
        checks.append(f"❌ Invalid action: {rule.get('action')}")
    
    # Print results
    for check in checks:
        print(check)
    
    # Summary
    failures = [c for c in checks if c.startswith("❌")]
    if not failures:
        print(f"\n🎉 Configuration is valid and ready for deployment!")
    else:
        print(f"\n⚠️  Found {len(failures)} issue(s) that need to be fixed")

validate_security_rule(admin_ssh_rule)

print("\n💡 In a real LangGraph application, this validation logic would be")
print("   implemented as a node in your graph, ensuring configurations are")
print("   correct before they're sent to the SCM API!")
print("\n📚 This pattern matches docs/examples/security_policy.py - you can")
print("   use client.security_rule.create(admin_ssh_rule, rulebase='pre')")
print("   to deploy this exact configuration to Strata Cloud Manager!")

---

## 7. Summary

Congratulations! You've completed the Type Annotations notebook. Let's recap what you've learned:

### What We Covered

1. **Python Dictionaries** - Flexible key-value data structures that are the foundation for TypedDict
   - Easy to use but lack type safety
   - Can lead to bugs in larger projects
   - Critical to understand for network automation

2. **TypedDict** - Structured dictionary schemas with type safety
   - Define exactly what keys and types are expected
   - Critical for LangGraph state management
   - Provides IDE autocomplete and error checking
   - **Applied to real SCM address objects and security rules**

3. **Union Types** - Accept multiple specific types
   - `Union[int, float]` means "either int or float"
   - Provides flexibility with type safety
   - Used extensively in LangChain and LangGraph

4. **Optional** - Handle nullable values
   - `Optional[str]` means "either str or None"
   - Perfect for fields that might not always be present
   - Still type-safe - can't be any random type

5. **Any Type** - Accept any type whatsoever
   - Use sparingly for truly dynamic data
   - Prefer specific types when possible
   - Good for external data without schemas

6. **Lambda Functions** - Inline anonymous functions
   - Shortcuts for simple operations
   - Efficient with `map()`, `filter()`, `sorted()`
   - **Used to filter and process security rules**

### Real-World Applications You Built

Throughout this notebook, you created practical, production-ready TypedDict schemas:

- ✅ **SCM Address Objects**: IP/Netmask, FQDN, and IP Range configurations  
  - Reference: [docs/examples/address_objects.py](../docs/examples/address_objects.py) lines 9-15, 21-26, 32-37
  - Ready to use with `client.address.create()`

- ✅ **Security Rules**: Complete firewall rule structures with logging
  - Reference: [docs/examples/security_policy.py](../docs/examples/security_policy.py) lines 9-20, 26-39
  - Ready to use with `client.security_rule.create(rule, rulebase='pre')`

- ✅ **Validation Functions**: Error-checking logic for configuration safety
  - Demonstrated with address configuration validation
  - Applied to security rule validation

- ✅ **Lambda Processing**: Filtering rules by action, tags, and zones
  - `filter()` to find specific rule types
  - `map()` to extract data from rules
  - `sorted()` to order rules by name
  - `Counter` to analyze rule distributions

These are the exact patterns you'll use when building LangGraph workflows for Palo Alto Networks automation!

### Why This Matters for LangGraph

These type annotations are fundamental to building LangGraph applications:
- **TypedDict** defines your agent's state schema
- **Union** and **Optional** handle flexible data types
- **Lambda** functions help transform and filter data efficiently
- **Validation** ensures configurations are correct before API calls

### Progressive Complexity Achieved

This notebook followed a clear learning path:

1. **Dictionaries** → Understanding the foundation
2. **TypedDict** → Adding structure and type safety
3. **Union/Optional** → Handling flexible types
4. **Any** → Understanding when to use dynamic typing
5. **Lambda** → Efficient data transformation
6. **Real SCM Examples** → Applying everything to production use cases

### Sneak Peak: What's Next in Notebook 102

Now that you've mastered TypedDict and SCM structures, get ready to bring them to life in LangGraph!

**In Notebook 102: Core Concepts, you'll learn:**

1. **State** - Using the TypedDict patterns you just learned to define what data flows through your graph
   - Example: `class AddressState(TypedDict): name: str, ip_netmask: str, status: str`
   
2. **Nodes** - Functions that process state (using the exact SCM patterns from this notebook!)
   - Example: A `validate_address()` node that checks IP/netmask format
   - Example: A `create_address()` node that calls `client.address.create()`
   
3. **Graphs** - Connecting nodes into automated workflows
   - Example: `validate → prepare → create → verify` pipeline for address objects
   
4. **Edges** - Defining how execution flows from node to node
   - Sequential flow: `graph.add_edge("validate", "create")`
   - START/END nodes for entry and exit points

**You'll build your first complete LangGraph application:**
- A 4-node graph that validates, prepares, creates, and verifies SCM address objects
- Using the exact `AddressObjectNetmask` TypedDict from this notebook
- Demonstrating how state accumulates data as it flows through nodes
- Showing real pan-scm-sdk integration patterns

**The exciting part?** Everything you learned in this notebook transfers directly:
- Your TypedDict schemas become LangGraph state
- Your validation functions become graph nodes  
- Your lambda filters become state transformations
- Your SCM knowledge becomes production automation!

**🚀 Ready to build real automation workflows? Let's move to Notebook 102!**

### Self-Assessment Checklist

Ready to move on to Notebook 102? Check your understanding:

- [ ] Can you explain why TypedDict is better than plain dictionaries for configuration automation?
- [ ] Can you create a TypedDict schema with both required and Optional fields?
- [ ] Do you understand how Union types allow accepting multiple specific types?
- [ ] Can you write lambda functions to use with map(), filter(), and sorted()?
- [ ] Can you build SCM address object and security rule TypedDict structures matching the pan-scm-sdk API?
- [ ] Do you understand how to validate configurations before deployment?

**If you checked all boxes, you're ready for Notebook 102: Core Concepts!**

If you're unsure about any topic, review that section before continuing. These foundations are essential for LangGraph success.

---

### Key Reminder

You don't need to memorize any of this - just have a high-level understanding of what these annotations are and why they're useful. You can always refer back to this notebook when you need a refresher!

### Reference Files for Your Projects

When building your own LangGraph workflows, reference these files for accurate SCM API structures:
- **Address Objects**: [docs/examples/address_objects.py](../docs/examples/address_objects.py)
- **Security Rules**: [docs/examples/security_policy.py](../docs/examples/security_policy.py)

**Ready to build your first LangGraph application? Let's move on to Notebook 102!**